In [1]:
import pandas as pd
import numpy as np
import xlrd
import glob
import os

import time

In [2]:
# this script parses important insurer information from an xlsx file 
# original data can be found here: https://www.cms.gov/apps/mlr/mlr-search.aspx

# author: Grace Guan
# date created: 11/24/17

In [10]:
ycoordlabels = ["1HealthInsuranceINDIVIDUALTotalasof12/31/14","2HealthInsuranceINDIVIDUALTotalasof3/31/15"
,"3HealthInsuranceINDIVIDUALDualContracts(IncludedinTotalasof3/31/15)","4HealthInsuranceINDIVIDUALDeferredPY1(Add)"
,"5HealthInsuranceINDIVIDUALDeferredCY(Subtract)","2AHealthInsuranceINDIVIDUAL[RiskCorridors]Totalasof3/31/15"
,"6HealthInsuranceSMALLGROUPTotalasof12/31/14","7HealthInsuranceSMALLGROUPTotalasof3/31/15"
,"8HealthInsuranceSMALLGROUPDualContracts(IncludedinTotalasof3/31/15)","9HealthInsuranceSMALLGROUPDeferredPY1(Add)"
,"10HealthInsuranceSMALLGROUPDeferredCY(Subtract)","7AHealthInsuranceSMALLGROUP[RiskCorridors]Totalasof3/31/15"
,"11HealthInsuranceLARGEGROUPTotalasof12/31/14","12HealthInsuranceLARGEGROUPTotalasof3/31/15"
,"13HealthInsuranceLARGEGROUPDualContracts(IncludedinTotalasof3/31/15)","14HealthInsuranceLARGEGROUPDeferredPY1(Add)"
,"15HealthInsuranceLARGEGROUPDeferredCY(Subtract)","16Mini-MedINDIVIDUALTotalasof12/31/14"
,"17Mini-MedINDIVIDUALTotalasof3/31/15","18Mini-MedINDIVIDUALDualContracts(IncludedinTotalasof3/31/15)"
,"19Mini-MedSMALLGROUPTotalasof12/31/14","20Mini-MedSMALLGROUPTotalasof3/31/15"
,"21Mini-MedSMALLGROUPDualContracts(IncludedinTotalasof3/31/15)","22Mini-MedLARGEGROUPTotalasof12/31/14"
,"23Mini-MedLARGEGROUPTotalasof3/31/15","24Mini-MedLARGEGROUPDualContracts(IncludedinTotalasof3/31/15)"
,"25ExpatSMALLGROUPTotalasof12/31/14","26ExpatSMALLGROUPTotalasof3/31/15"
,"27ExpatSMALLGROUPDualContracts(IncludedinTotalasof3/31/15)","28ExpatSMALLGROUPDeferredPY1(Add)"
,"29ExpatSMALLGROUPDeferredCY(Subtract)","30ExpatLARGEGROUPTotalasof12/31/14","31ExpatLARGEGROUPTotalasof3/31/15"
,"32ExpatLARGEGROUPDualContracts(IncludedinTotalasof3/31/15)","33ExpatLARGEGROUPDeferredPY1(Add)"
,"34ExpatLARGEGROUPDeferredCY(Subtract)","35StudentHealthINDIVIDUALTotalasof12/31/14"
,"36StudentHealthINDIVIDUALTotalasof3/31/15","37StudentHealthINDIVIDUALDualContracts(IncludedinTotalasof3/31/15)"
,"38StudentHealthINDIVIDUALDeferredPY1(Add)","39StudentHealthINDIVIDUALDeferredCY(Subtract)"
,"40GovernmentProgramPlansTotalasof12/31/14","41OtherHealthBusinessTotalasof12/31/14"
,"42Aggregate2%RuleTotalasof12/31/14","43UninsuredPlansTotalasof12/31/14","44GrandTotalTotalasof12/31/14"]
    
xcoordlabels1 = ["1.Premium",
"1.1Totaldirectpremiumearned",
"1.2Federalhighriskpools",
"1.3Statehighriskpools",
"1.4Netassumedlesscededreinsurancepremiumearned(excludeamountsalreadyreportedinLine1.1)",
"1.5OtheradjustmentsduetoMLRcalculations-premium",
"1.6Riskrevenue",
"2.Claims",
"2.1Totalincurredclaims(MLRFormPart2Line2.16)",
"2.2Prescriptiondrugs(informationalonly;alreadyincludedintotalincurredclaimsabove)",
"2.3Pharmaceuticalrebates(informationalonly;alreadyexcludedfromtotalincurredclaimsabove)",
"2.4Statestoplossmarketstabilizationandclaim/censusbasedassessments(informationalonly;alreadyexcludedfromtotalincurredclaimsabove)",
"2.5Netassumedlesscededclaimsincurred(excludeamountsalreadyreportedinLine2.1)",
"2.6OtheradjustmentsduetoMLRcalculations–claimsincurred",
"2.7Rebatespaid",
"2.8EstimatedrebatesunpaidattheendofthepreviousMLRreportingyear",
"2.9EstimatedrebatesunpaidattheendoftheMLRreportingyear",
"2.10Fee-for-serviceandco-payrevenue(netofexpenses)",
"2.11Allowablefraudreductionexpenses(MLRFormPart2Line2.17)",
"3.FederalandStateTaxesandLicensingorRegulatoryFees",
"3.1FederaltaxesandassessmentsincurredbythereportingissuerduringtheMLRreportingyear",
"3.1aFederalincometaxesdeductiblefrompremiuminMLRcalculations",
"3.1bPatientCenteredOutcomesResearchInstitute(PCORI)Fee",
"3.1cAffordableCareActsection9010Fee",
"3.1dOtherFederalTaxesandassessmentsdeductiblefrompremium",
"3.2StateinsurancepremiumandothertaxesincurredbythereportingissuerduringtheMLRreportingyear(deductiblefrompremiuminMLRcalculation)",
"3.2aStateincomeexcisebusinessandothertaxes",
"3.2bStatepremiumtaxes",
"3.2cCommunitybenefitexpendituresdeductiblefrompremiuminMLRcalculations",
"3.3Regulatoryauthoritylicensesandfees",
"3.3aFederalTransitionalReinsuranceProgramcontributions",
"3.3bOtherFederalandStateregulatoryauthoritylicensesandfees",
"4.HealthCareQualityImprovementExpensesIncurred",
"4.1Improvehealthoutcomes",
"4.2Activitiestopreventhospitalreadmission",
"4.3Improvepatientsafetyandreducemedicalerrors",
"4.4Wellnessandhealthpromotionactivities",
"4.5Healthinformationtechnologyexpensesrelatedtoimprovinghealthcarequality",
"5.Non-ClaimsCosts",
"5.1CostcontainmentexpensesnotincludedinqualityimprovementexpensesinSection4",
"5.2Allotherclaimsadjustmentexpenses",
"5.3Directsalessalariesandbenefits",
"5.4Agentsandbrokersfeesandcommissions",
"5.5Othertaxes",
"5.5aTaxesandassessments(excludeamountsreportedinSection3orLine9)",
"5.5bFinesandpenaltiesofregulatoryauthorities(excludeamountsreportedinLine3.3)",
"5.6Othergeneralandadministrativeexpenses",
"5.7Communitybenefitexpenditures(informationalonly;includeamountsreportedinLines3.2cand5.6)",
"6.Incomefromfeesofuninsuredplans",
"7.OtherIndicatorsorinformation",
"7.1Numberofpolicies/certificates",
"7.2Numberofcoveredlives",
"7.3Numberofgroups",
"7.4Membermonths",
"7.5Numberoflife-years",
"8.Netinvestmentincomeandothergain/(loss)",
"9.OtherFederalincometaxes(excludetaxesonLines3.1a-d)"]

xcoordlabels2 = ["1.Premium",
"1.1Directpremiumwritten",
"1.2Unearnedpremiumprioryear",
"1.3UnearnedpremiumMLRReportingyear",
"1.4Experienceratingrefunds(ratecredits)paid",
"1.4aExperienceratingrefundswithallincurreddatespaidintheMLRreportingyear",
"1.4bExperienceratingrefundsassociatedwithpremiumearnedonlyinthereportingyearandpaidthrough3/31ofthefollowingyear",
"1.5Reserveforexperienceratingrefunds(ratecredits)MLRReportingyear",
"1.6Reserveforexperienceratingrefunds(ratecredits)prioryear",
"1.7Premiumbalanceswrittenoff",
"1.8Groupconversioncharges",
"1.9FederalTransitionalReinsuranceProgrampaymentsexpectedfromHHS(asindicatedbyHHSasof6/30)",
"1.10FederalRiskAdjustmentProgramnetpaymentsexpectedfromHHS/(chargespayabletoHHS)(asindicatedbyHHSasof6/30)",
"1.11FederalRiskCorridorsProgramnetpayments/(charges)",
"1.12Premiumcededunder100%reinsurance(informationalonly;alreadyexcludedfromLines1.1-1.11)",
"1.13Premiumassumedunder100%reinsurance(informationalonly;alreadyincludedinLines1.1-1.11)",
"1.14AdvancepaymentsofthepremiumtaxcreditreceivedfromHHS(informationalonly;alreadyincludedinLines1.1-1.11)",
"2.Claims",
"2.1ClaimsPaid",
"2.1aClaimspaidduringtheMLRreportingyearregardlessofincurreddate",
"2.1bClaimsincurredonlyduringtheMLRreportingyearpaidthrough3/31ofthefollowingyear",
"2.2Directclaimliability",
"2.2aLiabilityasof12/31ofMLRreportingyearforallclaimsregardlessofincurreddate",
"2.2bLiabilityforclaimsincurredonlyduringtheMLRreportingyearcalculatedasof3/31ofthefollowingyear",
"2.3Directclaimliabilityprioryear",
"2.4Directclaimreserves",
"2.4aReservesasof12/31ofMLRreportingyearforallclaimsregardlessofincurreddate",
"2.4bReservesforclaimsincurredonlyduringtheMLRreportingyearcalculatedasof3/31ofthefollowingyear",
"2.5Directclaimreservesprioryear",
"2.6Directcontractreserves",
"2.6aDirectcontractreserves12/31column",
"2.6bDirectcontractreserves3/31dualcontractdeferredcolumns",
"2.7Directcontractreservesprioryear",
"2.8Experienceratingrefunds(ratecredits)paid",
"2.8aExperienceratingrefundswithallincurreddatespaidintheMLRreportingyear",
"2.8bExperienceratingrefundsassociatedwithpremiumearnedonlyinthereportingyearandpaidthrough3/31ofthefollowingyear",
"2.9Reserveforexperienceratingrefunds(ratecredits)",
"2.9aReservedinMLRreportingyearregardlessofincurreddate",
"2.9bReservesspecifictotheMLRreportingyearthrough3/31ofthefollowingyear",
"2.10Reserveforexperienceratingrefunds(ratecredits)prioryear",
"2.11Incurredmedicalincentivepoolandbonuses",
"2.11aPaidmedicalincentivepoolsandbonusesMLRReportingyear",
"2.11bAccruedmedicalincentivepoolsandbonusesMLRReportingyear",
"2.11cAccruedmedicalincentivepoolsandbonusesprioryear",
"2.12Nethealthcarereceivables",
"2.12aHealthcarereceivablesMLRReportingyear",
"2.12bHealthcarereceivablesprioryear",
"2.13Contingentbenefitandlawsuitreserves",
"2.14Groupconversioncharges",
"2.15Blendedrateadjustment",
"2.16Totalincurredclaims",
"2.17Allowablefraudreductionexpense(thesmallerofLines2.17aor2.17b)",
"2.17aTotalfraudreductionexpense",
"2.17bTotalfraudrecoveriesthatreducedpaidclaimsinLine2.1",
"2.18Advancepaymentsofcost-sharingreductions"]

xcoordspart0 = ["CompanyName:",
"GroupAffiliation:",
"FederalEIN:",
"A.M.BestNumber:",
"NAICGroupCode:",
"NAICCompanyCode:",
"DBA/MarketingName:",
"HIOSIssuerID:",
"BusinessintheStateof:",
"DomiciliaryState:",
"Address:",
"FederalTaxExempt:",
"Marketplace:",
"MergeMarkets-Ind/SmGrp:",
"Not-For-Profit:",
"MLRReportingYear:"]


labels = list()
    
for a in range(0, 16): # parse sheet 0
    labels.append(xcoordspart0[a])

for b in range(0, 57): # parse sheet 1
    for c in range(0, 46):
        labels.append(xcoordlabels1[b] + ycoordlabels[c])

for d in range(0, 55): # parse sheet 2
    for e in range (0, 46):
        labels.append(xcoordlabels2[d] + ycoordlabels[e])
        
#print(labels)

In [11]:
def parse_input(df16, xls):
    
    sheet0 = xls.parse(0)
    
    # 2014, 2015, and 2016 format
    if sheet0.iloc[2,0] != 'Company Name:': # should say "Company Name:"
        return
    
    reporting_year = sheet0.iloc[17,1]

    sheet1 = xls.parse(1)
    sheet2 = xls.parse(2)
    
    data = list()
    
    for a in range(2, 18): # parse sheet 0
        data.append(sheet0.iloc[a, 1])
    
    for b in range(2, 59): # parse sheet 1
        for c in range(2, 48):
            data.append(sheet1.iloc[b, c])
    
    for d in range(2, 57): # parse sheet 2
        for e in range (2, 48):
            data.append(sheet2.iloc[d, e])

    df16.loc[df16.shape[0]] =  data

In [14]:
# create the dataframe

t0 = time.time()

errorcount = 0

mydf16 = pd.DataFrame(columns=labels)

print(os.getcwd())
print(os.getcwd())
os.chdir('C:/Users/guanz/Desktop/2016_data_extracted') #2016_data_extracted
FileList = glob.glob('*.xls*')

FailedFileList = list()

for File in FileList:
    try:
        myxls = pd.ExcelFile(File)
        print(File + " success")
        parse_input(mydf16, myxls)
    except xlrd.XLRDError:
        print(File + " failure")
        errorcount = errorcount + 1
        FailedFileList.append(File)
        pass


print("There were " + str(errorcount) + " encrypted files.")
print(mydf16)


C:\Users\guanz\Desktop\2016_data_testrun
C:\Users\guanz\Desktop\2016_data_testrun
2016 -Companion MLR- Reporting-Form-Grand Total.xlsx success
2016 BCBSKS MLR_Template_Grand_Total.xlsx success
2016 BCBSKS MLR_Template_Kansas.xlsx success
2016 HHNJ MLR_Template_Grand_Total.xlsx success
2016 HHNJ MLR_Template_New_Jersey.xlsx success
2016 HHSI MLR_Template_Grand_Total.xlsx success
2016 HHSI MLR_Template_New_Jersey.xlsx success
2016 OHIC MLR_Template_Grand_Total.xlsx success
2016 OHIC MLR_Template_Virginia.xlsx success
2016 OHP MLR_Template_Grand_Total.xlsx success
2016 OHP MLR_Template_Virginia.xlsx success
2016 Solutions MLR_Template_Grand_Total.xlsx success
2016 Solutions MLR_Template_Kansas.xlsx success
2016-MLR-Reporting-Form-Alabama.xlsx success
2016-MLR-Reporting-Form-Arizona.xlsx success
2016-MLR-Reporting-Form-Arkansas.xlsx success
2016-MLR-Reporting-Form-California.xlsx success
2016-MLR-Reporting-Form-Connecticut.xlsx success
2016-MLR-Reporting-Form-Florida.xlsx success
2016-MLR-

Final_MLR_Template_Arkansas.xlsx success
Final_MLR_Template_Georgia.xlsx success
Final_MLR_Template_Grand_Total.xlsx success
Final_MLR_Template_Illinois.xlsx success
Final_MLR_Template_Indiana.xlsx success
Final_MLR_Template_Iowa.xlsx success
Final_MLR_Template_Kansas.xlsx success
Final_MLR_Template_Louisiana.xlsx success
Final_MLR_Template_Michigan.xlsx success
Final_MLR_Template_Minnesota.xlsx success
Final_MLR_Template_Mississippi.xlsx success
Final_MLR_Template_Missouri.xlsx success
Final_MLR_Template_Nebraska.xlsx success
Final_MLR_Template_Ohio.xlsx success
Final_MLR_Template_Pennsylvania.xlsx success
Final_MLR_Template_Tennessee.xlsx success
Final_MLR_Template_Texas.xlsx success
Final_MLR_Template_Virginia.xlsx success
Final_MLR_Template_Wisconsin.xlsx success
Final_PreferredOneCommunityHealthPlan_Grand_Total.xlsx success
Final_PreferredOneCommunityHealthPlan_Minnesota_96724.xlsx success
Final_PreferredOneInsuranceCompany_Grand_Total.xlsx success
Final_PreferredOneInsuranceCompa

MLR16_006_KIC_Hawaii.xlsx success
MLR16_006_KIC_Idaho.xlsx success
MLR16_006_KIC_Illinois.xlsx success
MLR16_006_KIC_Indiana.xlsx success
MLR16_006_KIC_Iowa.xlsx success
MLR16_006_KIC_Kansas.xlsx success
MLR16_006_KIC_Kentucky.xlsx success
MLR16_006_KIC_Louisiana.xlsx success
MLR16_006_KIC_Maine.xlsx success
MLR16_006_KIC_Maryland.xlsx success
MLR16_006_KIC_Massachusetts.xlsx success
MLR16_006_KIC_Michigan.xlsx success
MLR16_006_KIC_Minnesota.xlsx success
MLR16_006_KIC_Mississippi.xlsx success
MLR16_006_KIC_Missouri.xlsx success
MLR16_006_KIC_Montana.xlsx success
MLR16_006_KIC_Nebraska.xlsx success
MLR16_006_KIC_Nevada.xlsx success
MLR16_006_KIC_New_Hampshire.xlsx success
MLR16_006_KIC_New_Jersey.xlsx success
MLR16_006_KIC_New_Mexico.xlsx success
MLR16_006_KIC_New_York.xlsx success
MLR16_006_KIC_North_Carolina.xlsx success
MLR16_006_KIC_North_Dakota.xlsx success
MLR16_006_KIC_Ohio.xlsx success
MLR16_006_KIC_Oklahoma.xlsx success
MLR16_006_KIC_Oregon.xlsx success
MLR16_006_KIC_Pennsylva

MLR_Template_Alabama (7).xlsx success
MLR_Template_Alabama (8).xlsx success
MLR_Template_Alabama (9).xlsx success
MLR_Template_Alabama.xlsx success
MLR_Template_Alaska (10).xlsx success
MLR_Template_Alaska (11).xlsx success
MLR_Template_Alaska (12).xlsx success
MLR_Template_Alaska (13).xlsx success
MLR_Template_Alaska (14).xlsx success
MLR_Template_Alaska (15).xlsx success
MLR_Template_Alaska (16).xlsx success
MLR_Template_Alaska (17).xlsx success
MLR_Template_Alaska (18).xlsx success
MLR_Template_Alaska (19).xlsx success
MLR_Template_Alaska (2).xlsx success
MLR_Template_Alaska (20).xlsx success
MLR_Template_Alaska (21).xlsx success
MLR_Template_Alaska (22).xlsx success
MLR_Template_Alaska (3).xlsx success
MLR_Template_Alaska (4).xlsx success
MLR_Template_Alaska (5).xlsx success
MLR_Template_Alaska (6).xlsx success
MLR_Template_Alaska (7).xlsx success
MLR_Template_Alaska (8).xlsx success
MLR_Template_Alaska (9).xlsx success
MLR_Template_Alaska.xlsx success
MLR_Template_American_Samoa.x

MLR_Template_CGLIC_Grand_Total.xlsx success
MLR_Template_CGLIC_Guam.xlsx success
MLR_Template_CGLIC_Hawaii.xlsx success
MLR_Template_CGLIC_Idaho.xlsx success
MLR_Template_CGLIC_Illinois.xlsx success
MLR_Template_CGLIC_Indiana.xlsx success
MLR_Template_CGLIC_Iowa.xlsx success
MLR_Template_CGLIC_Kansas.xlsx success
MLR_Template_CGLIC_Kentucky.xlsx success
MLR_Template_CGLIC_Louisiana.xlsx success
MLR_Template_CGLIC_Maine.xlsx success
MLR_Template_CGLIC_Maryland.xlsx success
MLR_Template_CGLIC_Massachusetts.xlsx success
MLR_Template_CGLIC_Michigan.xlsx success
MLR_Template_CGLIC_Minnesota.xlsx success
MLR_Template_CGLIC_Mississippi.xlsx success
MLR_Template_CGLIC_Missouri.xlsx success
MLR_Template_CGLIC_Montana.xlsx success
MLR_Template_CGLIC_Nebraska.xlsx success
MLR_Template_CGLIC_Nevada.xlsx success
MLR_Template_CGLIC_New_Hampshire.xlsx success
MLR_Template_CGLIC_New_Jersey.xlsx success
MLR_Template_CGLIC_New_Mexico.xlsx success
MLR_Template_CGLIC_New_York.xlsx success
MLR_Template_CGL

...

MLR_Template_New_York (2).xlsx success
MLR_Template_New_York (20).xlsx success
MLR_Template_New_York (21).xlsx success
MLR_Template_New_York (22).xlsx success
MLR_Template_New_York (23).xlsx success
MLR_Template_New_York (24).xlsx success
MLR_Template_New_York (25).xlsx success
MLR_Template_New_York (26).xlsx success
MLR_Template_New_York (27).xlsx success
MLR_Template_New_York (28).xlsx success
MLR_Template_New_York (29).xlsx success
MLR_Template_New_York (3).xlsx success
MLR_Template_New_York (30).xlsx success
MLR_Template_New_York (31).xlsx success
MLR_Template_New_York (32).xlsx success
MLR_Template_New_York (33).xlsx success
MLR_Template_New_York (34).xlsx success
MLR_Template_New_York (35).xlsx success
MLR_Template_New_York (36).xlsx success
MLR_Template_New_York (37).xlsx success
MLR_Template_New_York (38).xlsx success
MLR_Template_New_York (39).xlsx success
MLR_Template_New_York (4).xlsx success
MLR_Template_New_York (40).xlsx success
MLR_Template_New_York (41).xlsx success
MLR

MLR_Template_Oklahoma (12).xlsx success
MLR_Template_Oklahoma (13).xlsx success
MLR_Template_Oklahoma (14).xlsx success
MLR_Template_Oklahoma (15).xlsx success
MLR_Template_Oklahoma (16).xlsx success
MLR_Template_Oklahoma (17).xlsx success
MLR_Template_Oklahoma (18).xlsx success
MLR_Template_Oklahoma (19).xlsx success
MLR_Template_Oklahoma (2).xlsx success
MLR_Template_Oklahoma (20).xlsx success
MLR_Template_Oklahoma (21).xlsx success
MLR_Template_Oklahoma (22).xlsx success
MLR_Template_Oklahoma (23).xlsx success
MLR_Template_Oklahoma (24).xlsx success
MLR_Template_Oklahoma (25).xlsx success
MLR_Template_Oklahoma (26).xlsx success
MLR_Template_Oklahoma (27).xlsx success
MLR_Template_Oklahoma (28).xlsx success
MLR_Template_Oklahoma (29).xlsx success
MLR_Template_Oklahoma (3).xlsx success
MLR_Template_Oklahoma (30).xlsx success
MLR_Template_Oklahoma (31).xlsx success
MLR_Template_Oklahoma (32).xlsx success
MLR_Template_Oklahoma (33).xlsx success
MLR_Template_Oklahoma (34).xlsx success
ML

MLR_Template_South_Carolina (15).xlsx success
MLR_Template_South_Carolina (16).xlsx success
MLR_Template_South_Carolina (17).xlsx success
MLR_Template_South_Carolina (18).xlsx success
MLR_Template_South_Carolina (19).xlsx success
MLR_Template_South_Carolina (2).xlsx success
MLR_Template_South_Carolina (20).xlsx success
MLR_Template_South_Carolina (21).xlsx success
MLR_Template_South_Carolina (22).xlsx success
MLR_Template_South_Carolina (23).xlsx success
MLR_Template_South_Carolina (24).xlsx success
MLR_Template_South_Carolina (25).xlsx success
MLR_Template_South_Carolina (26).xlsx success
MLR_Template_South_Carolina (27).xlsx success
MLR_Template_South_Carolina (28).xlsx success
MLR_Template_South_Carolina (29).xlsx success
MLR_Template_South_Carolina (3).xlsx success
MLR_Template_South_Carolina (30).xlsx success
MLR_Template_South_Carolina (31).xlsx success
MLR_Template_South_Carolina (32).xlsx success
MLR_Template_South_Carolina (33).xlsx success
MLR_Template_South_Carolina (34).xls

MLR_Template_Texas (66).xlsx success
MLR_Template_Texas (67).xlsx success
MLR_Template_Texas (68).xlsx success
MLR_Template_Texas (69).xlsx success
MLR_Template_Texas (7).xlsx success
MLR_Template_Texas (70).xlsx success
MLR_Template_Texas (71).xlsx success
MLR_Template_Texas (72).xlsx success
MLR_Template_Texas (73).xlsx success
MLR_Template_Texas (74).xlsx success
MLR_Template_Texas (8).xlsx success
MLR_Template_Texas (9).xlsx success
MLR_Template_Texas.xlsx success
MLR_Template_Texas_2016_WCL.xlsx success
MLR_Template_Texas_20170726v3.xlsx success
MLR_Template_Texas_27248_CommunityHealthChoice.xlsx success
MLR_Template_Texas_ICSW_2016.xlsx success
MLR_Template_Texas_SWHP.xlsx success
MLR_Template_United_States_Virgin_Islands (2).xlsx success
MLR_Template_United_States_Virgin_Islands (3).xlsx success
MLR_Template_United_States_Virgin_Islands (4).xlsx success
MLR_Template_United_States_Virgin_Islands (5).xlsx success
MLR_Template_United_States_Virgin_Islands (6).xlsx success
MLR_Templ

MLR_Template_West_Virginia (11).xlsx success
MLR_Template_West_Virginia (12).xlsx success
MLR_Template_West_Virginia (13).xlsx success
MLR_Template_West_Virginia (14).xlsx success
MLR_Template_West_Virginia (15).xlsx success
MLR_Template_West_Virginia (16).xlsx success
MLR_Template_West_Virginia (17).xlsx success
MLR_Template_West_Virginia (18).xlsx success
MLR_Template_West_Virginia (19).xlsx success
MLR_Template_West_Virginia (2).xlsx success
MLR_Template_West_Virginia (20).xlsx success
MLR_Template_West_Virginia (21).xlsx success
MLR_Template_West_Virginia (22).xlsx success
MLR_Template_West_Virginia (23).xlsx success
MLR_Template_West_Virginia (24).xlsx success
MLR_Template_West_Virginia (25).xlsx success
MLR_Template_West_Virginia (26).xlsx success
MLR_Template_West_Virginia (27).xlsx success
MLR_Template_West_Virginia (28).xlsx success
MLR_Template_West_Virginia (29).xlsx success
MLR_Template_West_Virginia (3).xlsx success
MLR_Template_West_Virginia (30).xlsx success
MLR_Template

In [15]:

t1 = time.time()

print(t1-t0)

6693.104544878006


In [18]:
os.chdir('C:/Users/guanz/Desktop/JIW2/Week6') 
mydf16.to_csv("2016-all.csv")